In [ ]:
import requests
import json
import pandas as pd
from pandas import json_normalize
from google.colab import files
from datetime import datetime

token='0b8e6db6509f2bf80bda79ad71f5a6b6'
AV_token=""


def get_project_id(proj_name,i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getProjects&token={}".format(i)
    response = requests.get(url).json()
    ID_df = json_normalize(response)
    proj_id = ID_df.iloc[list(ID_df.iloc[:,1]).index(proj_name),0]
    return proj_id
#test1 = get_project_id("LG.com_UK")

def get_search_engines_id(proj_id,mytoken=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getSearchEngines&token={}&projectId={}".format(mytoken,proj_id)
    response = requests.get(url).json()
    se_df = json_normalize(response)
    id = list(se_df['id']).pop()
    return id

def get_search_engines_name(proj_id,mytoken=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getSearchEngines&token={}&projectId={}".format(mytoken,proj_id)
    response = requests.get(url).json()
    se_df = json_normalize(response)
    id = list(se_df['name']).pop()
    return id

def get_keyword_group_df(proj_id=111, i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getKeywordGroups&token={}&projectId={}".format(i,proj_id)
    response = requests.get(url).json()
    group_df = json_normalize(response)
    return group_df
#test2 = getkeyword_group_df(test1)

def get_keyword_group_id(group_name, proj_id=111, i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getKeywordGroups&token={}&projectId={}".format(i,proj_id)
    response = requests.get(url).json()
    group_df = json_normalize(response)
    group_id = group_df.iloc[list(group_df.iloc[:,1]).index(group_name),0]
    return group_id
#test3 = get_keyword_group_id('Brand',test1)

def get_metadata_df(proj_id,group_id,obj_date,i=token):
    meta_df = pd.DataFrame({})
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getMetadata&token={}&projectId={}&searchEngineId={}&keywordGroupId={}&startDate={}&stopDate={}&mode=plain".format(i,proj_id,-1,group_id,obj_date,obj_date)
    response = requests.get(url).text
    domain_list_raw = response.split('<br/>')
    colum_name = domain_list_raw[0].split(',')
    columindex=0
    for x in colum_name:
        flashdata= []
        for i in domain_list_raw[1:-1]:
            flashdata.append(i.split(',')[columindex])
        meta_df[x]=flashdata
        columindex = columindex+1
    return meta_df
#test4 = get_metadata_df(proj_id='111',group_id='33',obj_date='2021-03-29',i=token)
#test4 = get_metadata_df(test1,test3,'2021-03-30')

def get_domain_urls(proj_id,group_id,obj_date,obj_domain,i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getDomainUrls&token={}&projectId={}&searchEngineId=-1&keywordGroupId={}&date={}&domain={}&mode=plain".format(i,proj_id,group_id,obj_date,obj_domain)
    response = requests.get(url).text
    url_list_raw = response.split('<br/>')
    colum_name = url_list_raw[0].split(',')
    url_index=colum_name.index('url')
    colum_count=len(colum_name)
    url_df = pd.DataFrame(columns = colum_name)
    for i in url_list_raw[1:-1]:
        flashdata= []
        row_list = i.split(',')
        flashdata = flashdata+row_list[:url_index]
        flashdata.append(','.join(row_list[url_index:url_index-colum_count+1]))
        flashdata = flashdata+row_list[url_index-colum_count+1:]
        url_df = url_df.append(pd.Series(flashdata,index=url_df.columns), ignore_index=True)
    url_df=url_df.astype({'market share':'float','estimated visits':'float','click share':'float'})
    return url_df
#test5 = get_domain_urls(proj_id='111',group_id='33',obj_date='2021-03-29',obj_domain='lg.com',i=token)


def export_top_sites(av_proj_name,start,stop,searchE, num_url=12,token = AV_token):
    proj_name = '+'.join(av_proj_name.split())
    url = "https://api.awrcloud.com/v2/get.php?action=topsites_export&project={}&token={}&startDate={}&stopDate={}&pixelPosition=false&topUrls={}&searchEngine={}".format(proj_name, token, start, stop,num_url,searchE)
    response1 = requests.get(url).json()
    res_df = json_normalize(response1)
    down_url = res_df['details'][0]
    return down_url


In [ ]:
start_date = "2021-04-25"   
end_date = "2021-06-01" 
from datetime import datetime
import pandas as pd

def date_range(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    dates = [date.strftime("%Y-%m-%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates

date_range(start_date,end_date)

In [ ]:
# # 입력부
path = '/content/sample_data'
date_list = pd.date_range('2021-05-24', periods=7)
fail_df= pd.DataFrame({},columns =['f_project','f_group','f_date'])
proj_list = ['US_TONE Free','ES_TONE Free','MS_XBOOM']
for xxx in proj_list:
    proj_id_re = get_project_id(xxx+'_Google')
    print(proj_id_re)
    group_df = get_keyword_group_df(proj_id_re)
    is_general_cate = group_df['name'] == 'General - Category'
    group_select = group_df[is_general_cate]
    group_select.reset_index(inplace=True)
    group_select
    group_name = group_select['name']
    group_id = group_select['id']
    for x in group_id[:]:
        all_df = pd.DataFrame({})
        count = 0
        filename = "{}_result_{}".format(xxx,x)
        for date in date_list:
            select_date = str(date)[:10]
            print(select_date)
            try:
                sub_df= get_metadata_df(proj_id = proj_id_re,group_id = x, obj_date = str(select_date),i=token)
                sub_df = sub_df.astype({'urls': 'float'})
            except:
                flash_data = [xxx, x, select_date]
                fail_df = fail_df.append(pd.Series(flash_data, index=fail_df.columns), ignore_index=True)
                print(fail_df)
                sub_df = pd.DataFrame({})
                pass
            all_df = pd.concat([all_df,sub_df])
        grouped_df = all_df['urls'].groupby(all_df['domain'])
        aa = grouped_df.sum()
        bb = aa.sort_values(ascending=False)
        count=count+1
        bb.to_excel('{}.xlsx'.format(filename),index=True)
        print(count)
    files.download('{}.xlsx'.format(filename))
    print(fail_df)